# Image to Matrix

In [1]:
import os
from tqdm import tqdm 
import cv2
import numpy as np
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
class_names = ['EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL']
nb_classes = len(class_names)

In [ ]:
def load_data():

    datasets = [ '/content/drive/MyDrive/WBC Dataset/TEST' ] 
    # /content/drive/MyDrive/WBC Dataset/TEST
    images = []
    labels = []

    # iterate through training and test sets
    for dataset in datasets:

        # iterate through folders in each dataset
        for folder in os.listdir(dataset):

            if folder in ['EOSINOPHIL']: label = 0
            elif folder in ['LYMPHOCYTE']: label = 1
            elif folder in ['MONOCYTE']: label = 2
            elif folder in ['NEUTROPHIL']: label = 3

            # iterate through each image in folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):

                # get pathname of each image
                img_path = os.path.join(os.path.join(dataset, folder), file)

                # Open and resize the| img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
                # image = cv2.resize(image, image_size)
                
                # src = cv2.imread(img_path)
                # tmp = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
                # _,alpha = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY)
                # b, g, r = cv2.split(src)
                # rgba = [b,g,r, alpha]
                # image = cv2.merge(rgba,4)
                # image = cv2.resize(image, image_size)

                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)

    images = np.array(images, dtype = 'float32')
    labels = np.array(labels, dtype = 'int32')

    return images, labels

In [ ]:
images, labels = load_data()


100%|██████████| 48/48 [00:10<00:00,  4.66it/s]


In [ ]:
images.shape

(71, 240, 320)

In [ ]:
labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3], dtype=int32)

In [ ]:
# # from numpy import asarray
# import numpy as np
# # load the image
# image = Image.open('_0_187.jpeg')
# # convert image to numpy array
# data = np.array(image)
# print(type(data))
# # summarize shape
# print(data.shape)


# # # create Pillow image
# # image2 = Image.fromarray(data)
# # print(type(image2))

# # # summarize image details
# # print(image2.mode)
# # print(image2.size)

In [ ]:
  
# data

In [ ]:
def threshold(matrix):
  thresh = 160
  for i in range (len(matrix)):
    for j in range (len(matrix[0])):
      x = matrix[i][j]
      if x > thresh:
        matrix[i][j] = 1
      else:
        matrix[i][j] = 0
        

def sgn(z):
  if z > 0 :
    return 1
  elif z < 0 :
    return -1
  else :
    return 0

def edge (new_matrix,old_matrix,grad_matrix):
  t=0
  dt=0.5
  while (t<2):
    for i in range (len(old_matrix)):
      for j in range (len(old_matrix[0])):
        if i == 0 or j == 0 or i == len(old_matrix)-1 or j == len(old_matrix[0])-1:
          continue
        grad_matrix[i][j] = -np.sqrt((old_matrix[i+1][j] - old_matrix[i][j])**2 + 
        (old_matrix[i][j+1] - old_matrix[i][j])**2) * sgn(old_matrix[i+1][j] - 2*old_matrix[i][j] + old_matrix[i-1][j] + 
                                                  old_matrix[i][j+1] - 2*old_matrix[i][j] + old_matrix[i][j-1])
        new_matrix[i][j] = old_matrix[i][j]+dt* grad_matrix[i][j]
    t=t+dt

In [ ]:
def abc ():
  new_matrixes = []
  # grad_matrixes = []
  fix_matrixes = []
  for i in range (len(labels)):
    print('Iteration',i)
    # rgb_weights = [0.2989, 0.5870, 0.1140]
    # grayscale_image = np.dot(images[i], rgb_weights)
    # matrix = grayscale_image.copy()
    # threshold(matrix)
    old_matrix = images[i].copy()
    new_matrix = images[i].copy()
    grad_matrix = images[i].copy()
    edge (new_matrix,old_matrix,grad_matrix)
    new_matrixes.append(new_matrix)
    # grad_matrixes.append(grad_matrix)
    fix_matrix = new_matrix.copy()
    threshold(fix_matrix)
    fix_matrixes.append(fix_matrix)

  
  return new_matrixes, fix_matrixes

In [ ]:
new_matrixes, fix_matrixes = abc()

In [ ]:
nsamples, nx, ny = images.shape
print (nsamples,nx,ny)
images = images.reshape((nsamples,nx*ny))

In [ ]:
fix_matrixes = np.array(fix_matrixes)

In [ ]:
nsamples, nx, ny = fix_matrixes.shape
print (nsamples,nx,ny)
fix_matrixes = fix_matrixes.reshape((nsamples,nx*ny))

In [ ]:
new_matrixes = np.array(new_matrixes)

In [ ]:
nsamples, nx, ny = new_matrixes.shape
print (nsamples,nx,ny)
new_matrixes = new_matrixes.reshape((nsamples,nx*ny))

images

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.20)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)

print('Accuracy of K Nearest Neighbor :',accuracy_score(y_test, knn_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)
print('Accuracy of Random Forest :',accuracy_score(y_test, rfc_pred))

FIX MATRIX

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier



In [ ]:
scaler = RobustScaler()


In [ ]:
fix_matrixes = scaler.fit_transform(fix_matrixes)

In [ ]:
for i in range(30):
  X_trainF, X_testF, y_trainF, y_testF = train_test_split(fix_matrixes, labels, test_size=0.20)

  knn = KNeighborsClassifier(n_neighbors=9)
  knn.fit(X_trainF, y_trainF)
  knn_predF = knn.predict(X_testF)
  # print (i)
  # print('-----------------------------------------------------------------')
  print(i,'. Accuracy of K Nearest Neighbor :',accuracy_score(y_testF, knn_predF))

  # rfc = RandomForestClassifier(n_estimators=100)
  # rfc.fit(X_trainF, y_trainF)
  # rfc_predF = rfc.predict(X_testF)
  # print('Accuracy of Random Forest :',accuracy_score(y_testF, rfc_predF))
  # print('-----------------------------------------------------------------')

0 . Accuracy of K Nearest Neighbor : 0.32304038004750596
1 . Accuracy of K Nearest Neighbor : 0.32304038004750596
2 . Accuracy of K Nearest Neighbor : 0.332541567695962
3 . Accuracy of K Nearest Neighbor : 0.32304038004750596
4 . Accuracy of K Nearest Neighbor : 0.33016627078384797
5 . Accuracy of K Nearest Neighbor : 0.342042755344418
6 . Accuracy of K Nearest Neighbor : 0.33729216152019004
7 . Accuracy of K Nearest Neighbor : 0.3657957244655582
8 . Accuracy of K Nearest Neighbor : 0.33729216152019004
9 . Accuracy of K Nearest Neighbor : 0.36342042755344417
10 . Accuracy of K Nearest Neighbor : 0.3752969121140142
11 . Accuracy of K Nearest Neighbor : 0.3586698337292161
12 . Accuracy of K Nearest Neighbor : 0.37292161520190026
13 . Accuracy of K Nearest Neighbor : 0.3586698337292161
14 . Accuracy of K Nearest Neighbor : 0.332541567695962
15 . Accuracy of K Nearest Neighbor : 0.342042755344418
16 . Accuracy of K Nearest Neighbor : 0.32304038004750596
17 . Accuracy of K Nearest Neighbor 

NEW MATRIX

In [ ]:
from sklearn.model_selection import train_test_split
X_trainN, X_testN, y_trainN, y_testN = train_test_split(new_matrixes, labels, test_size=0.20)


In [ ]:


knn = KNeighborsClassifier(n_neighbors=75)
knn.fit(X_trainN, y_trainN)
knn_predN = knn.predict(X_testN)

print('Accuracy of K Nearest Neighbor :',accuracy_score(y_testN, knn_predN))

In [ ]:
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_trainN, y_trainN)
rfc_predN = rfc.predict(X_testN)
print('Accuracy of Random Forest :',accuracy_score(y_testN, rfc_predN))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


for i in range(30):
  X_trainF, X_testF, y_trainF, y_testF = train_test_split(new_matrixes, labels, test_size=0.20)

  knn = KNeighborsClassifier(n_neighbors=1 )
  knn.fit(X_trainF, y_trainF)
  knn_predF = knn.predict(X_testF)
  # print (i)
  print('-----------------------------------------------------------------')
  print(i,'. Accuracy of K Nearest Neighbor :',accuracy_score(y_testF, knn_predF))

  rfc = RandomForestClassifier(n_estimators=100)
  rfc.fit(X_trainF, y_trainF)
  rfc_predF = rfc.predict(X_testF)
  print('Accuracy of Random Forest :',accuracy_score(y_testF, rfc_predF))
  print('-----------------------------------------------------------------')


-----------------------------------------------------------------
0 . Accuracy of K Nearest Neighbor : 0.27790973871733965
-----------------------------------------------------------------
-----------------------------------------------------------------
1 . Accuracy of K Nearest Neighbor : 0.3087885985748218
-----------------------------------------------------------------
-----------------------------------------------------------------
2 . Accuracy of K Nearest Neighbor : 0.29691211401425177
-----------------------------------------------------------------
-----------------------------------------------------------------
3 . Accuracy of K Nearest Neighbor : 0.334916864608076
-----------------------------------------------------------------
-----------------------------------------------------------------
4 . Accuracy of K Nearest Neighbor : 0.2992874109263658
-----------------------------------------------------------------
-----------------------------------------------------------